##Prepare a Euro 2024 Group Stage and knockout stage with random goal values with less number of tables as possible.

In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/teamseuro", recurse=True)



Out[495]: True

In [0]:
%sql
 DROP TABLE IF EXISTS TeamsEuro;
 Drop TABLE IF EXISTS ThirdPlacementCombinations;
 Drop TABLE IF EXISTS EliminationMatches;

 

In [0]:
%sql
-- Create Teams table
CREATE OR REPLACE TABLE TeamsEuro (
    TeamID INT,
    TeamName STRING,
    GroupName STRING
);

-- Insert data into Teams table
INSERT INTO TeamsEuro VALUES
    (1, 'France', 'A'),
    (2, 'Germany', 'A'),
    (3, 'Spain', 'A'),
    (4, 'Italy', 'A'),
    (5, 'Portugal', 'B'),
    (6, 'England', 'B'),
    (7, 'Netherlands', 'B'),
    (8, 'Belgium', 'B'),
    (9, 'Croatia', 'C'),
    (10, 'Poland', 'C'),
    (11, 'Switzerland', 'C'),
    (12, 'Turkey', 'C'),
    (13, 'Sweden', 'D'),
    (14, 'Denmark', 'D'),
    (15, 'Russia', 'D'),
    (16, 'Austria', 'D'),
    (17, 'Ukraine', 'E'),
    (18, 'Wales', 'E'),
    (19, 'Scotland', 'E'),
    (20, 'Czech Republic', 'E'),
    (21, 'Slovakia', 'F'),
    (22, 'Hungary', 'F'),
    (23, 'Finland', 'F'),
    (24, 'North Macedonia', 'F');


num_affected_rows,num_inserted_rows
24,24


In [0]:
%sql
select * from TeamsEuro

TeamID,TeamName,GroupName
1,France,A
2,Germany,A
3,Spain,A
4,Italy,A
5,Portugal,B
6,England,B
7,Netherlands,B
8,Belgium,B
9,Croatia,C
10,Poland,C


##Creating a table for group stage matches with a random goals.and declaring a winner 

In [0]:
%sql
-- Create GroupStageMatches table
CREATE OR REPLACE TABLE GroupStageMatche AS
SELECT
    t1.TeamID AS Team1ID,
    t1.TeamName AS Team1Name,
    t1.GroupName AS Team1Group,
    t2.TeamID AS Team2ID,
    t2.TeamName AS Team2Name,
    t2.GroupName AS Team2Group,
    ROUND(RAND() * 5) AS Team1Goals,  -- Random goal value for Team1
    ROUND(RAND() * 5) AS Team2Goals,   -- Random goal value for Team2
    CASE
        WHEN Team1Goals > Team2Goals THEN t1.TeamName
        WHEN Team2Goals > Team1Goals THEN t2.TeamName
        ELSE "Draw"
    END AS Winner
FROM
    TeamsEuro t1
CROSS JOIN
    TeamsEuro t2
WHERE
    t1.GroupName = t2.GroupName
    AND t1.TeamID < t2.TeamID;  -- Avoid duplicate matches and self-matches


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from GroupStageMatche

Team1ID,Team1Name,Team1Group,Team2ID,Team2Name,Team2Group,Team1Goals,Team2Goals,Winner
1,France,A,4,Italy,A,4.0,2.0,France
1,France,A,3,Spain,A,4.0,2.0,France
1,France,A,2,Germany,A,4.0,4.0,Draw
2,Germany,A,4,Italy,A,2.0,0.0,Germany
2,Germany,A,3,Spain,A,2.0,3.0,Spain
3,Spain,A,4,Italy,A,5.0,4.0,Spain
5,Portugal,B,8,Belgium,B,4.0,3.0,Portugal
5,Portugal,B,7,Netherlands,B,4.0,0.0,Portugal
5,Portugal,B,6,England,B,4.0,4.0,Draw
6,England,B,8,Belgium,B,2.0,5.0,Belgium


##Create a group stage summary tie sheet , with team point , ranking and goal difference 

In [0]:
%sql
-- Create GroupStageSummary table with Team Rank and Tie-breaking on Goal Difference
CREATE OR REPLACE TABLE GroupStageSummary AS
WITH MatchResults AS (
    SELECT
        Team1ID AS TeamID,
        Team1Name AS TeamName,
        Team1Group AS GroupName,
        Team1Goals,
        Team2ID,
        Team2Name,
        Team2Group,
        Team2Goals,
        Winner
    FROM
        GroupStageMatche

    UNION ALL

    SELECT
        Team2ID AS TeamID,
        Team2Name AS TeamName,
        Team2Group AS GroupName,
        Team2Goals,
        Team1ID,
        Team1Name,
        Team1Group,
        Team1Goals,
        Winner
    FROM
        GroupStageMatche
)
SELECT
    TeamID,
    TeamName,
    GroupName,

    DENSE_RANK() OVER (PARTITION BY GroupName ORDER BY
        SUM(CASE WHEN Winner = TeamName THEN 3 WHEN Winner = 'Draw' THEN 1 ELSE 0 END) DESC,
        SUM(Team1Goals - Team2Goals) DESC,
        SUM(Team1Goals) DESC,
        SUM(Team2Goals) ASC,
        COUNT(*) DESC
    ) AS TeamRank,
    COUNT(*) AS MatchesPlayed,
    SUM(CASE WHEN Winner = TeamName THEN 1 ELSE 0 END) AS MatchesWon,
    SUM(CASE WHEN Winner != TeamName AND Winner != 'Draw' THEN 1 ELSE 0 END) AS MatchesLost,
    SUM(CASE WHEN Winner = 'Draw' THEN 1 ELSE 0 END) AS MatchesDrawn,
    SUM(CASE WHEN Winner = TeamName THEN 3 WHEN Winner = 'Draw' THEN 1 ELSE 0 END) AS TotalPoints,
    SUM(Team1Goals - Team2Goals) AS GoalDifference,
    SUM(Team1Goals) AS GoalScored,
    SUM(Team2Goals) AS GoalConceled
   
FROM MatchResults
GROUP BY TeamID, TeamName, GroupName
ORDER BY GroupName, TeamRank;


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from groupstagesummary

TeamID,TeamName,GroupName,TeamRank,MatchesPlayed,MatchesWon,MatchesLost,MatchesDrawn,TotalPoints,GoalDifference,GoalScored,GoalConceled
1,France,A,1,3,2,0,1,7,4.0,12.0,8.0
3,Spain,A,2,3,2,1,0,6,0.0,10.0,10.0
2,Germany,A,3,3,1,1,1,4,1.0,8.0,7.0
4,Italy,A,4,3,0,3,0,0,-5.0,6.0,11.0
5,Portugal,B,1,3,2,0,1,7,5.0,12.0,7.0
8,Belgium,B,2,3,2,1,0,6,3.0,11.0,8.0
6,England,B,3,3,1,1,1,4,-1.0,10.0,11.0
7,Netherlands,B,4,3,0,3,0,0,-7.0,4.0,11.0
11,Switzerland,C,1,3,2,1,0,6,3.0,8.0,5.0
12,Turkey,C,2,3,1,1,1,4,-1.0,8.0,9.0


##Selection of 12 winners i e top 2 team of every group stage , and query for remaining third rank teams

In [0]:
%sql
-- Create Round of 16 (Ro16) table
CREATE OR REPLACE TABLE RoundOf16 AS
WITH GroupStandings AS (
    SELECT
        TeamID,
        TeamName,
        GroupName,
        TeamRank,
        TotalPoints,
        GoalDifference,
        GoalScored,
        GoalConceled,
        ROW_NUMBER() OVER (PARTITION BY GroupName ORDER BY TeamRank) AS RowInGroup
    FROM
        GroupStageSummary
)
SELECT
    TeamID,
    TeamName,
    TeamRank,   GroupName,
    CONCAT(GroupName, TeamRank) AS GroupRank,
    TotalPoints,
    GoalDifference,
    GoalScored,
    GoalConceled
FROM
    GroupStandings
WHERE
    RowInGroup <= 2
UNION ALL
SELECT
    TeamID,
    TeamName,
    TeamRank,   GroupName,
    CONCAT(GroupName, TeamRank) AS GroupRank,
    TotalPoints,
    GoalDifference,
    GoalScored,
    GoalConceled
FROM (
    SELECT
        gs.TeamID,
        gs.TeamName,
        gs.TeamRank,
        gs.TotalPoints,
        gs.GoalDifference,
        gs.GoalScored,
        gs.GoalConceled,
        ROW_NUMBER() OVER (ORDER BY gs.TotalPoints DESC, gs.GoalDifference DESC, gs.GoalScored DESC, gs.GoalConceled DESC) AS RowOverall,
        gs.GroupName  -- Include GroupName in the subquery
    FROM
        GroupStandings gs
    WHERE
        gs.RowInGroup = 3
) AS ThirdPlaceTeams
WHERE
    RowOverall <= 4
ORDER BY
    TeamID;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM RoundOf16


TeamID,TeamName,TeamRank,GroupName,GroupRank,TotalPoints,GoalDifference,GoalScored,GoalConceled
1,France,1,A,A1,7,4.0,12.0,8.0
2,Germany,3,A,A3,4,1.0,8.0,7.0
3,Spain,2,A,A2,6,0.0,10.0,10.0
5,Portugal,1,B,B1,7,5.0,12.0,7.0
6,England,3,B,B3,4,-1.0,10.0,11.0
8,Belgium,2,B,B2,6,3.0,11.0,8.0
9,Croatia,3,C,C3,4,-1.0,5.0,6.0
11,Switzerland,1,C,C1,6,3.0,8.0,5.0
12,Turkey,2,C,C2,4,-1.0,8.0,9.0
13,Sweden,1,D,D1,7,6.0,10.0,4.0


In [0]:
%sql
SELECT
    CONCAT_WS('', COLLECT_LIST(SUBSTRING(GroupName, 1) || TeamRank)) AS FinalCombination
FROM
    RoundOf16
WHERE
    TeamRank = 3;

FinalCombination
A3B3C3F3


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/thirdplacementcombinations", recurse=True)

Out[506]: False

##Combination sequence table for thirdplacement .

In [0]:
%sql
CREATE or replace TABLE ThirdPlacementCombinations(
  SId INT,
  QualificationSequence VARCHAR(8),
  B1 CHAR(2),
  C1 CHAR(2),
  E1 CHAR(2),
  F1 CHAR(2)
);

INSERT INTO  ThirdPlacementCombinations (SId, QualificationSequence, B1, C1, E1, F1)
VALUES
    (1, 'A3B3C3D3', 'A3', 'D3', 'B3', 'C3'),
    (2, 'A3B3C3E3', 'A3', 'E3', 'B3', 'C3'),
    (3, 'A3B3C3F3', 'A3', 'F3', 'B3', 'C3'),
    (4, 'A3B3D3E3', 'D3', 'E3', 'A3', 'B3'),
    (5, 'A3B3D3F3', 'D3', 'F3', 'A3', 'B3'),
    (6, 'A3B3E3F3', 'E3', 'F3', 'B3', 'A3'),
    (7, 'A3C3D3E3', 'E3', 'D3', 'C3', 'A3'),
    (8, 'A3C3D3F3', 'F3', 'D3', 'C3', 'A3'),
    (9, 'A3C3E3F3', 'E3', 'F3', 'C3', 'A3'),
    (10, 'A3D3E3F3', 'E3', 'F3', 'D3', 'A3'),
    (11, 'B3C3D3E3', 'E3', 'D3', 'B3', 'C3'),
    (12, 'B3C3D3F3', 'F3', 'D3', 'C3', 'B3'),
    (13, 'B3C3E3F3', 'F3', 'E3', 'C3', 'B3'),
    (14, 'B3D3E3F3', 'F3', 'E3', 'D3', 'B3'),
    (15, 'C3D3E3F3', 'F3', 'E3', 'D3', 'C3');

SELECT * FROM ThirdPlacementCombinations;

SId,QualificationSequence,B1,C1,E1,F1
1,A3B3C3D3,A3,D3,B3,C3
2,A3B3C3E3,A3,E3,B3,C3
3,A3B3C3F3,A3,F3,B3,C3
4,A3B3D3E3,D3,E3,A3,B3
5,A3B3D3F3,D3,F3,A3,B3
6,A3B3E3F3,E3,F3,B3,A3
7,A3C3D3E3,E3,D3,C3,A3
8,A3C3D3F3,F3,D3,C3,A3
9,A3C3E3F3,E3,F3,C3,A3
10,A3D3E3F3,E3,F3,D3,A3


In [0]:
%sql
-- %sql
-- WITH FirstQuery AS (
--     SELECT
--         CONCAT_WS('', COLLECT_LIST(SUBSTRING(GroupName, 1) || TeamRank)) AS DynamicValue
--     FROM
--         RoundOf16
--     WHERE
--         TeamRank = 3
-- )

-- -- Compare with the provided table
-- SELECT *
-- FROM ThirdPlacementCombinations
-- WHERE QualificationSequence = (SELECT DynamicValue FROM FirstQuery);

##Combination pattern  for Top 4 teams ranking 3 , who have qualified 

In [0]:
%sql

With QualificationCompare AS (
  SELECT c.SId, c.B1, c.C1, c.E1, c.F1, pc.PlacementCombination 
  FROM ThirdPlacementCombinations c
  JOIN (
    SELECT
        CONCAT_WS('', COLLECT_LIST(SUBSTRING(GroupName, 1) || TeamRank)) AS PlacementCombination
    FROM
        RoundOf16
    WHERE
        TeamRank = 3
  ) pc
  ON c.QualificationSequence = pc.PlacementCombination
)
  SELECT * FROM QualificationCompare;

SId,B1,C1,E1,F1,PlacementCombination
3,A3,F3,B3,C3,A3B3C3F3


In [0]:
# %python
# dbutils.fs.rm("dbfs:/user/hive/warehouse/eliminationmatches", recurse=True)

In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/eliminationmatches", recurse=True)

Out[510]: False

##Table creation for elimination matches

In [0]:
%sql
CREATE OR REPLACE TABLE EliminationMatches (
    MatchID INT,
    Team1 VARCHAR(53),
    Team1Goals INT,
    Team2Goals INT,
    Team2 VARCHAR(53),
    MatchWinner VARCHAR(53),
    MatchStage VARCHAR(20)
);

##ROund of 16 matches between those selected teams .

In [0]:
%sql

With QualificationCompare AS (
  SELECT c.SId, c.B1, c.C1, c.E1, c.F1, pc.PlacementCombination 
  FROM ThirdPlacementCombinations c
  JOIN (
    SELECT
        CONCAT_WS('', COLLECT_LIST(SUBSTRING(GroupName, 1) || TeamRank)) AS PlacementCombination
    FROM
        RoundOf16
    WHERE
        TeamRank = 3
  ) pc
  ON c.QualificationSequence = pc.PlacementCombination
)
  
INSERT INTO EliminationMatches (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 
      CASE 
        WHEN t1.TeamRank = 1 AND t1.GroupName = 'B' THEN 1
        WHEN t1.TeamRank = 1 AND t1.GroupName = 'A' THEN 2
        WHEN t1.TeamRank = 1 AND t1.GroupName = 'F' THEN 3
        WHEN t1.TeamRank = 2 AND t1.GroupName = 'D' THEN 4
        WHEN t1.TeamRank = 1 AND t1.GroupName = 'E' THEN 5
        WHEN t1.TeamRank = 1 AND t1.GroupName = 'D' THEN 6
        WHEN t1.TeamRank = 1 AND t1.GroupName = 'C' THEN 7
        WHEN t1.TeamRank = 2 AND t1.GroupName = 'A' THEN 8
 
      END
    ) AS MatchID,
    t1.TeamName AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    t2.TeamName AS Team2,
    CASE
        WHEN Team1Goals > Team2Goals THEN t1.TeamName
        WHEN Team2Goals > Team1Goals THEN t2.TeamName
        ELSE
          CASE
            WHEN Team1 < Team2 THEN Team1
            ELSE Team2
          END
    END AS MatchWinner,
    'Round of 16' AS MatchStage

FROM RoundOf16 t1
JOIN RoundOf16 t2
ON (
     (t1.GroupRank = 'B1' AND t2.GroupRank = (SELECT B1 FROM QualificationCompare)) OR
    (t1.GroupRank = 'A1' AND t2.GroupRank = 'C2') OR
    (t1.GroupRank = 'F1' AND t2.GroupRank = (SELECT F1 FROM QualificationCompare)) OR
    (t1.GroupRank = 'D2' AND t2.GroupRank = 'E2') OR
    (t1.GroupRank = 'E1' AND t2.GroupRank = (SELECT E1 FROM QualificationCompare)) OR
    (t1.GroupRank = 'D1' AND t2.GroupRank = 'F2') OR
    (t1.GroupRank = 'C1' AND t2.GroupRank = (SELECT C1 FROM QualificationCompare)) OR
    (t1.GroupRank = 'A2' AND t2.GroupRank = 'B2')
);


num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
select * from eliminationmatches

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
1,Portugal,2,5,Germany,Germany,Round of 16
2,France,3,5,Turkey,Turkey,Round of 16
3,North Macedonia,2,1,Croatia,North Macedonia,Round of 16
4,Russia,2,1,Wales,Russia,Round of 16
5,Czech Republic,1,2,England,England,Round of 16
6,Sweden,5,5,Finland,Finland,Round of 16
7,Switzerland,0,3,Slovakia,Slovakia,Round of 16
8,Spain,1,4,Belgium,Belgium,Round of 16


##Quater Final Matches

In [0]:
%sql


INSERT INTO EliminationMatches (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 8 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    t2.MatchWinner AS Team2,
    'TBD' AS MatchWinner,
    'Quater Finals' AS MatchStage

FROM EliminationMatches t1
CROSS JOIN EliminationMatches t2
ON
  (t1.MatchID = 1 AND t2.MatchId = 2) OR
  (t1.MatchID = 3 AND t2.MatchId = 4) OR
  (t1.MatchID = 5 AND t2.MatchId = 6) OR
  (t1.MatchID = 7 AND t2.MatchId = 8);

UPDATE EliminationMatches
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'Quater Finals';

num_affected_rows
4


In [0]:
%sql
select * from eliminationmatches

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
1,Portugal,2,5,Germany,Germany,Round of 16
2,France,3,5,Turkey,Turkey,Round of 16
3,North Macedonia,2,1,Croatia,North Macedonia,Round of 16
4,Russia,2,1,Wales,Russia,Round of 16
5,Czech Republic,1,2,England,England,Round of 16
6,Sweden,5,5,Finland,Finland,Round of 16
7,Switzerland,0,3,Slovakia,Slovakia,Round of 16
8,Spain,1,4,Belgium,Belgium,Round of 16
9,Germany,6,6,Turkey,Germany,Quater Finals
10,North Macedonia,3,1,Russia,North Macedonia,Quater Finals


##Semi-Final Matches

In [0]:
%sql
INSERT INTO EliminationMatches (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 12 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    t2.MatchWinner AS Team2,
    'TBD' AS MatchWinner,
    'Semifinals' AS MatchStage

FROM EliminationMatches t1
CROSS JOIN EliminationMatches t2
ON
  (t1.MatchID = 9 AND t2.MatchId = 10) OR
  (t1.MatchID = 11 AND t2.MatchId = 12);

UPDATE EliminationMatches
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'Semifinals';


num_affected_rows
2


In [0]:
%sql
select * from EliminationMatches

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
1,Portugal,2,5,Germany,Germany,Round of 16
2,France,3,5,Turkey,Turkey,Round of 16
3,North Macedonia,2,1,Croatia,North Macedonia,Round of 16
4,Russia,2,1,Wales,Russia,Round of 16
5,Czech Republic,1,2,England,England,Round of 16
6,Sweden,5,5,Finland,Finland,Round of 16
7,Switzerland,0,3,Slovakia,Slovakia,Round of 16
8,Spain,1,4,Belgium,Belgium,Round of 16
9,Germany,6,6,Turkey,Germany,Quater Finals
10,North Macedonia,3,1,Russia,North Macedonia,Quater Finals


##Final Match, to declare winner of Euro cup 

In [0]:
%sql
INSERT INTO EliminationMatches (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 14 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    t2.MatchWinner AS Team2,
    'TBD' AS MatchWinner,
    'Final' AS MatchStage

FROM EliminationMatches t1
CROSS JOIN EliminationMatches t2
ON
  (t1.MatchID = 13 AND t2.MatchId = 14);

UPDATE EliminationMatches
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'Final';


num_affected_rows
1


In [0]:
%sql
select * from EliminationMatches

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
1,Portugal,2,5,Germany,Germany,Round of 16
2,France,3,5,Turkey,Turkey,Round of 16
3,North Macedonia,2,1,Croatia,North Macedonia,Round of 16
4,Russia,2,1,Wales,Russia,Round of 16
5,Czech Republic,1,2,England,England,Round of 16
6,Sweden,5,5,Finland,Finland,Round of 16
7,Switzerland,0,3,Slovakia,Slovakia,Round of 16
8,Spain,1,4,Belgium,Belgium,Round of 16
9,Germany,6,6,Turkey,Germany,Quater Finals
10,North Macedonia,3,1,Russia,North Macedonia,Quater Finals


In [0]:
%sql
select * from EliminationMatches

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
1,Portugal,2,5,Germany,Germany,Round of 16
2,France,3,5,Turkey,Turkey,Round of 16
3,North Macedonia,2,1,Croatia,North Macedonia,Round of 16
4,Russia,2,1,Wales,Russia,Round of 16
5,Czech Republic,1,2,England,England,Round of 16
6,Sweden,5,5,Finland,Finland,Round of 16
7,Switzerland,0,3,Slovakia,Slovakia,Round of 16
8,Spain,1,4,Belgium,Belgium,Round of 16
9,Germany,6,6,Turkey,Germany,Quater Finals
10,North Macedonia,3,1,Russia,North Macedonia,Quater Finals


##Matches for thirdplace  playoff

In [0]:
%sql
-- Inserting Third Place Playoff
INSERT INTO EliminationMatches (MatchID, Team1, Team1Goals, Team2Goals, Team2, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 16 AS MatchID,
    CASE
        WHEN t1.Team1Goals < t1.Team2Goals THEN t1.Team1
        ELSE t1.Team2
    END AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    ROUND(RAND() * 6) AS Team2Goals,
    CASE
        WHEN t2.Team1Goals < t2.Team2Goals THEN t2.Team1
        ELSE t2.Team2
    END AS Team2,
    'TBD' AS MatchWinner,
    '3rd Place' AS MatchStage
FROM EliminationMatches t1
JOIN EliminationMatches t2 ON (t1.MatchID = 13 AND t2.MatchId = 14);

-- Updating Third Place Playoff Winner
UPDATE EliminationMatches
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = '3rd Place';


num_affected_rows
1


In [0]:
%sql
select * from EliminationMatches

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
1,Portugal,2,5,Germany,Germany,Round of 16
2,France,3,5,Turkey,Turkey,Round of 16
3,North Macedonia,2,1,Croatia,North Macedonia,Round of 16
4,Russia,2,1,Wales,Russia,Round of 16
5,Czech Republic,1,2,England,England,Round of 16
6,Sweden,5,5,Finland,Finland,Round of 16
7,Switzerland,0,3,Slovakia,Slovakia,Round of 16
8,Spain,1,4,Belgium,Belgium,Round of 16
9,Germany,6,6,Turkey,Germany,Quater Finals
10,North Macedonia,3,1,Russia,North Macedonia,Quater Finals


## Table Visualization

In [0]:
#import libraries
import pandas as pd
from pyspark.sql import SparkSession
#Spark Session
spark = SparkSession.builder \
    .appName("Visualization") \
    .getOrCreate()

In [0]:
groupstage = spark.sql("SELECT * FROM groupstagesummary")
gs_pd = groupstage.toPandas()
gs_pd = gs_pd.applymap(lambda x: str(x).split(".")[0])

display(gs_pd)

TeamID,TeamName,GroupName,TeamRank,MatchesPlayed,MatchesWon,MatchesLost,MatchesDrawn,TotalPoints,GoalDifference,GoalScored,GoalConceled
1,France,A,1,3,2,0,1,7,4,12,8
3,Spain,A,2,3,2,1,0,6,0,10,10
2,Germany,A,3,3,1,1,1,4,1,8,7
4,Italy,A,4,3,0,3,0,0,-5,6,11
5,Portugal,B,1,3,2,0,1,7,5,12,7
8,Belgium,B,2,3,2,1,0,6,3,11,8
6,England,B,3,3,1,1,1,4,-1,10,11
7,Netherlands,B,4,3,0,3,0,0,-7,4,11
11,Switzerland,C,1,3,2,1,0,6,3,8,5
12,Turkey,C,2,3,1,1,1,4,-1,8,9


In [0]:

#GroupStage Visualization Table
euro_vis = pd.DataFrame({
    'Group': gs_pd['GroupName'],
    'TeamRank':gs_pd['TeamRank'],
    'Country': gs_pd['TeamName'],
    'P': gs_pd['MatchesPlayed'],
    'W': gs_pd['MatchesWon'],
    'D': gs_pd['MatchesDrawn'],
    'L': gs_pd['MatchesLost'],
    'GS': gs_pd['GoalScored'],
    'GC': gs_pd['GoalConceled'],
    'GD': gs_pd['GoalDifference'],
    'Pts': gs_pd['TotalPoints']
})
     

In [0]:
mtgs_row_pd = pd.DataFrame({col: [''] for col in euro_vis.columns})
     

In [0]:
left_gs = euro_vis[euro_vis['Group'].str.contains('^[ABC]')]

right_gs = euro_vis[euro_vis['Group'].str.contains('^[DEF]')]

#reseting index for both df
left_gs.reset_index(drop=True, inplace=True)
right_gs.reset_index(drop=True, inplace=True)

display(left_gs)
display(right_gs)

Group,TeamRank,Country,P,W,D,L,GS,GC,GD,Pts
A,1,France,3,2,1,0,12,8,4,7
A,2,Spain,3,2,0,1,10,10,0,6
A,3,Germany,3,1,1,1,8,7,1,4
A,4,Italy,3,0,0,3,6,11,-5,0
B,1,Portugal,3,2,1,0,12,7,5,7
B,2,Belgium,3,2,0,1,11,8,3,6
B,3,England,3,1,1,1,10,11,-1,4
B,4,Netherlands,3,0,0,3,4,11,-7,0
C,1,Switzerland,3,2,0,1,8,5,3,6
C,2,Turkey,3,1,1,1,8,9,-1,4


Group,TeamRank,Country,P,W,D,L,GS,GC,GD,Pts
D,1,Sweden,3,2,1,0,10,4,6,7
D,2,Russia,3,1,1,1,5,5,0,4
D,3,Austria,3,1,0,2,4,6,-2,3
D,4,Denmark,3,1,0,2,4,8,-4,3
E,1,Czech Republic,3,3,0,0,9,2,7,9
E,2,Wales,3,2,0,1,9,7,2,6
E,3,Ukraine,3,1,0,2,7,8,-1,3
E,4,Scotland,3,0,0,3,4,12,-8,0
F,1,North Macedonia,3,2,1,0,6,2,4,7
F,2,Finland,3,2,1,0,10,7,3,7


In [0]:
#list of groupstage according to groupname
left_gs_grouped = [left_gs.iloc[i:i+4] for i in range(0, len(left_gs), 4)]

#iterating over above list and adding empty row except for last one
for i in range(len(left_gs_grouped)):
    if i < len(left_gs_grouped) - 1:
        left_gs_grouped[i] = pd.concat([left_gs_grouped[i], mtgs_row_pd,mtgs_row_pd], ignore_index=True)

#joining them back after adding empty rows
left_gs_spaced = pd.concat(left_gs_grouped).reset_index(drop=True)

lgs_new_column_name ={'Group': 'L_Group',
                      'TeamRank': 'L_TeamRank',
                       'Country': 'L_Country',
                       'P': 'L_P',
                       'W': 'L_W',
                       'D': 'L-D',
                       'L': 'L_L',
                       'GS': 'L_GS',
                       'GC': 'L_GC',
                       'GD': 'L_GD',
                       'Pts': 'L_Pts'
                       }
left_gs_spaced = left_gs_spaced.rename(columns=lgs_new_column_name)


right_gs_grouped = [right_gs.iloc[i:i+4] for i in range(0, len(right_gs), 4)]

#iterating over above list and adding empty row except for last one
for i in range(len(right_gs_grouped)):
    if i < len(right_gs_grouped) - 1:
        right_gs_grouped[i] = pd.concat([right_gs_grouped[i], mtgs_row_pd,mtgs_row_pd], ignore_index=True)

#joining them back after adding empty rows
right_gs_spaced = pd.concat(right_gs_grouped).reset_index(drop=True)

rgs_new_column_name ={'Group': 'R_Group',
                      'TeamRank': 'R_TeamRank',
                       'Country': 'R_Country',
                       'P': 'R_P',
                       'W': 'R_W',
                       'D': 'R-D',
                       'L': 'R_L',
                       'GS': 'R_GS',
                       'GC': 'R_GC',
                       'GD': 'R_GD',
                       'Pts': 'R_Pts'
                       }
right_gs_spaced = right_gs_spaced.rename(columns=rgs_new_column_name)


display(left_gs_spaced)
display(right_gs_spaced)
     

L_Group,L_TeamRank,L_Country,L_P,L_W,L-D,L_L,L_GS,L_GC,L_GD,L_Pts
A,1,France,3,2,1,0,12,8,4,7
A,2,Spain,3,2,0,1,10,10,0,6
A,3,Germany,3,1,1,1,8,7,1,4
A,4,Italy,3,0,0,3,6,11,-5,0
,,,,,,,,,,
,,,,,,,,,,
B,1,Portugal,3,2,1,0,12,7,5,7
B,2,Belgium,3,2,0,1,11,8,3,6
B,3,England,3,1,1,1,10,11,-1,4
B,4,Netherlands,3,0,0,3,4,11,-7,0


R_Group,R_TeamRank,R_Country,R_P,R_W,R-D,R_L,R_GS,R_GC,R_GD,R_Pts
D,1,Sweden,3,2,1,0,10,4,6,7
D,2,Russia,3,1,1,1,5,5,0,4
D,3,Austria,3,1,0,2,4,6,-2,3
D,4,Denmark,3,1,0,2,4,8,-4,3
,,,,,,,,,,
,,,,,,,,,,
E,1,Czech Republic,3,3,0,0,9,2,7,9
E,2,Wales,3,2,0,1,9,7,2,6
E,3,Ukraine,3,1,0,2,7,8,-1,3
E,4,Scotland,3,0,0,3,4,12,-8,0


## Creating Round of 16 table 

In [0]:

#getting data from sql table and into pandas dataframe
round_16 = spark.sql("SELECT * FROM EliminationMatches WHERE MatchStage = 'Round of 16'")
round_16_pd = round_16_pd.applymap(lambda x: str(x).split(".")[0])

round_16_pd = round_16.toPandas()
display(round_16_pd)
     
     

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
1,Portugal,2,5,Germany,Germany,Round of 16
2,France,3,5,Turkey,Turkey,Round of 16
3,North Macedonia,2,1,Croatia,North Macedonia,Round of 16
4,Russia,2,1,Wales,Russia,Round of 16
5,Czech Republic,1,2,England,England,Round of 16
6,Sweden,5,5,Finland,Finland,Round of 16
7,Switzerland,0,3,Slovakia,Slovakia,Round of 16
8,Spain,1,4,Belgium,Belgium,Round of 16


In [0]:

#storing required data in df
euro_r16_vis = pd.DataFrame(columns=['R16_Teams', 'R16_Score'])

#displaying all teams and respective scores in columns
for index, row in round_16_pd.iterrows():
    team1 = row['Team1']
    team1goals = row['Team1Goals']
    team2 = row['Team2']
    team2goals = row['Team2Goals']
    euro_r16_vis = pd.concat([euro_r16_vis, pd.DataFrame({'R16_Score': [team1goals], 'R16_Teams': [team1]})], ignore_index=True)
    euro_r16_vis = pd.concat([euro_r16_vis, pd.DataFrame({'R16_Score': [team2goals], 'R16_Teams': [team2]})], ignore_index=True)

display(euro_r16_vis)


R16_Teams,R16_Score
Portugal,2
Germany,5
France,3
Turkey,5
North Macedonia,2
Croatia,1
Russia,2
Wales,1
Czech Republic,1
England,2


In [0]:
# calculating midpoint of the dataFrame
splitingteams = len(euro_r16_vis) // 2

# splitting dataFrame into two equal halves
left_r16 = euro_r16_vis.iloc[:splitingteams]
right_r16 = euro_r16_vis.iloc[splitingteams:]

# index reset
left_r16.reset_index(drop=True, inplace=True)
right_r16.reset_index(drop=True, inplace=True)

# displaying the split DataFrames
display(left_r16)
display(right_r16)

R16_Teams,R16_Score
Portugal,2
Germany,5
France,3
Turkey,5
North Macedonia,2
Croatia,1
Russia,2
Wales,1


R16_Teams,R16_Score
Czech Republic,1
England,2
Sweden,5
Finland,5
Switzerland,0
Slovakia,3
Spain,1
Belgium,4


In [0]:

#empty row for round of 16
r16_row_pd = pd.DataFrame({col: [''] for col in euro_r16_vis.columns})

In [0]:
left_r16_grouped = [left_r16.iloc[i:i+2] for i in range(0, len(left_r16), 2)]

for i in range(len(left_r16_grouped)):
    if i < len(left_r16_grouped):
        left_r16_grouped[i] = pd.concat([r16_row_pd,left_r16_grouped[i], r16_row_pd], ignore_index=True)

left_r16_spaced = pd.concat(left_r16_grouped).reset_index(drop=True)

lr16_new_column_name ={'R16_Teams': 'L_R16_Teams',
                       'R16_Score': 'L_R16_Score'
                       }
left_r16_spaced = left_r16_spaced.rename(columns=lr16_new_column_name)

display(left_r16_spaced)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:467: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected bytes, got a 'int' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


L_R16_Teams,L_R16_Score
,
Portugal,2
Germany,5
,
,
France,3
Turkey,5
,
,
North Macedonia,2


In [0]:
right_r16_grouped = [right_r16.iloc[i:i+2] for i in range(0, len(right_r16), 2)]

for i in range(len(right_r16_grouped)):
    if i < len(right_r16_grouped):
        right_r16_grouped[i] = pd.concat([r16_row_pd,right_r16_grouped[i], r16_row_pd], ignore_index=True)

right_r16_spaced = pd.concat(right_r16_grouped).reset_index(drop=True)

rr16_new_column_name ={'R16_Teams': 'R_R16_Teams',
                       'R16_Score': 'R_R16_Score'
                       }
right_r16_spaced = right_r16_spaced.rename(columns=rr16_new_column_name)

rr16_new_column_order = ['R_R16_Score', 'R_R16_Teams']
right_r16_spaced = right_r16_spaced[rr16_new_column_order]

display(right_r16_spaced)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:467: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected bytes, got a 'int' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


R_R16_Score,R_R16_Teams
,
1,Czech Republic
2,England
,
,
5,Sweden
5,Finland
,
,
0,Switzerland


In [0]:

#getting data from sql table and into pandas dataframe
quaterFinals = spark.sql("SELECT * FROM EliminationMatches WHERE MatchStage = 'Quater Finals'")
pd_quaterfinals= quaterFinals.toPandas()
pd_quaterfinals = pd_quaterfinals.applymap(lambda x: str(x).split(".")[0])
display(pd_quaterfinals)
     
     

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
9,Germany,6,6,Turkey,Germany,Quater Finals
10,North Macedonia,3,1,Russia,North Macedonia,Quater Finals
11,England,5,1,Finland,England,Quater Finals
12,Slovakia,2,3,Belgium,Belgium,Quater Finals


In [0]:

#storing required data in df
qf_vis = pd.DataFrame(columns=['QF_Teams', 'QF_Score'])

#displaying all teams and respective scores in columns
for index, row in pd_quaterfinals.iterrows():
    team1 = row['Team1']
    team1goals = row['Team1Goals']
    team2 = row['Team2']
    team2goals = row['Team2Goals']
    qf_vis = pd.concat([qf_vis, pd.DataFrame({'QF_Teams': [team1], 'QF_Score': [team1goals]})], ignore_index=True)
    qf_vis = pd.concat([qf_vis, pd.DataFrame({'QF_Teams': [team2], 'QF_Score': [team2goals]})], ignore_index=True)

display(qf_vis)

QF_Teams,QF_Score
Germany,6
Turkey,6
North Macedonia,3
Russia,1
England,5
Finland,1
Slovakia,2
Belgium,3


In [0]:

# calculating midpoint of the dataFrame
midpoint_index2 = len(qf_vis) // 2

# splitting dataFrame into two equal halves
left_qf = qf_vis.iloc[:midpoint_index2]
right_qf = qf_vis.iloc[midpoint_index2:]

# index reset
left_qf.reset_index(drop=True, inplace=True)
right_qf.reset_index(drop=True, inplace=True)

# displaying the split DataFrames
display(left_qf)
display(right_qf)
     

QF_Teams,QF_Score
Germany,6
Turkey,6
North Macedonia,3
Russia,1


QF_Teams,QF_Score
England,5
Finland,1
Slovakia,2
Belgium,3


In [0]:
qf_row_pd = pd.DataFrame({col: [''] for col in qf_vis.columns})

left_qf_grouped = [left_qf.iloc[i:i+2] for i in range(0, len(left_qf), 2)]

for i in range(len(left_qf_grouped)):
    if i < len(left_qf_grouped):
        left_qf_grouped[i] = pd.concat([qf_row_pd, qf_row_pd, qf_row_pd,qf_row_pd, left_qf_grouped[i]], ignore_index=True)

left_qf_spaced = pd.concat(left_qf_grouped).reset_index(drop=True)

for _ in range(4):
    left_qf_spaced = pd.concat([left_qf_spaced, qf_row_pd], ignore_index=True)

lqf_new_column_name ={'QF_Teams': 'L_QF_Teams',
                       'QF_Score': 'L_QF_Score'
                       }
left_qf_spaced = left_qf_spaced.rename(columns=lqf_new_column_name)

display(left_qf_spaced)

L_QF_Teams,L_QF_Score
,
,
,
,
Germany,6
Turkey,6
,
,
,
,


In [0]:

right_qf_grouped = [right_qf.iloc[i:i+2] for i in range(0, len(left_qf), 2)]

for i in range(len(right_qf_grouped)):
    if i < len(right_qf_grouped):
        right_qf_grouped[i] = pd.concat([qf_row_pd, qf_row_pd, qf_row_pd,qf_row_pd, right_qf_grouped[i]], ignore_index=True)

right_qf_spaced = pd.concat(right_qf_grouped).reset_index(drop=True)

for _ in range(4):
    right_qf_spaced = pd.concat([right_qf_spaced, qf_row_pd], ignore_index=True)

rqf_new_column_name ={'QF_Teams': 'R_QF_Teams',
                       'QF_Score': 'R_QF_Score'
                       }
right_qf_spaced = right_qf_spaced.rename(columns=rqf_new_column_name)

rqf_new_column_order = ['R_QF_Score', 'R_QF_Teams']
right_qf_spaced = right_qf_spaced[rqf_new_column_order]

display(right_qf_spaced)

R_QF_Score,R_QF_Teams
,
,
,
,
5,England
1,Finland
,
,
,
,


In [0]:

sf = spark.sql("SELECT * FROM EliminationMatches WHERE MatchStage = 'Semifinals'")
sf_pd = sf.toPandas()
sf_pd = sf_pd.applymap(lambda x: str(x).split(".")[0])

display(sf_pd)
     

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
13,Germany,3,1,North Macedonia,Germany,Semifinals
14,England,2,6,Belgium,Belgium,Semifinals


In [0]:

#storing required data in df
sf_vis = pd.DataFrame(columns=['SF_Teams', 'SF_Score'])

#displaying all teams and respective scores in columns
for index, row in sf_pd.iterrows():
    team1 = row['Team1']
    team1goals = row['Team1Goals']
    team2 = row['Team2']
    team2goals = row['Team2Goals']
    sf_vis = pd.concat([sf_vis, pd.DataFrame({'SF_Teams': [team1], 'SF_Score': [team1goals]})], ignore_index=True)
    sf_vis = pd.concat([sf_vis, pd.DataFrame({'SF_Teams': [team2], 'SF_Score': [team2goals]})], ignore_index=True)

display(sf_vis)

SF_Teams,SF_Score
Germany,3
North Macedonia,1
England,2
Belgium,6


In [0]:

# calculating midpoint of the dataFrame
midpoint_index2 = len(sf_vis) // 2

# splitting dataFrame into two equal halves
left_sf = sf_vis.iloc[:midpoint_index2]
right_sf = sf_vis.iloc[midpoint_index2:]

# index reset
left_sf.reset_index(drop=True, inplace=True)
right_sf.reset_index(drop=True, inplace=True)

# displaying the split DataFrames
display(left_sf)
display(right_sf)

SF_Teams,SF_Score
Germany,3
North Macedonia,1


SF_Teams,SF_Score
England,2
Belgium,6


In [0]:
sf_row_pd = pd.DataFrame({col: [''] for col in sf_vis.columns})
left_sf_grouped = [left_sf.iloc[i:i+1] for i in range(0, len(left_sf), 1)]

for i in range(len(left_sf_grouped)):
    if i < len(left_sf_grouped):
        left_sf_grouped[i] = pd.concat([left_sf_grouped[i], sf_row_pd, sf_row_pd], ignore_index=True)

left_sf_spaced = pd.concat(left_sf_grouped).reset_index(drop=True)

for _ in range(6):
    left_sf_spaced = pd.concat([sf_row_pd, left_sf_spaced], ignore_index=True)

for _ in range(4):
    left_sf_spaced = pd.concat([left_sf_spaced, sf_row_pd], ignore_index=True)

lsf_new_column_name ={'SF_Teams': 'L_SF_Teams',
                       'SF_Score': 'L_SF_Score'
                       }

left_sf_spaced = left_sf_spaced.rename(columns=lsf_new_column_name)

display(left_sf_spaced)

L_SF_Teams,L_SF_Score
,
,
,
,
,
,
Germany,3
,
,
North Macedonia,1


In [0]:
right_sf_grouped = [right_sf.iloc[i:i+1] for i in range(0, len(right_sf), 1)]

for i in range(len(right_sf_grouped)):
    if i < len(right_sf_grouped):
        right_sf_grouped[i] = pd.concat([right_sf_grouped[i], sf_row_pd, sf_row_pd], ignore_index=True)

right_sf_spaced = pd.concat(right_sf_grouped).reset_index(drop=True)

for _ in range(6):
    right_sf_spaced = pd.concat([sf_row_pd, right_sf_spaced], ignore_index=True)

for _ in range(4):
    right_sf_spaced = pd.concat([right_sf_spaced, sf_row_pd], ignore_index=True)

rsf_new_column_name ={'SF_Teams': 'R_SF_Teams',
                       'SF_Score': 'R_SF_Score'
                       }

right_sf_spaced = right_sf_spaced.rename(columns=rsf_new_column_name)

rsf_new_column_order = ['R_SF_Score', 'R_SF_Teams']
right_sf_spaced = right_sf_spaced[rsf_new_column_order]

display(right_sf_spaced)

R_SF_Score,R_SF_Teams
,
,
,
,
,
,
2,England
,
,
6,Belgium


In [0]:

fin = spark.sql("SELECT * FROM EliminationMatches WHERE MatchStage = 'Final'")
fin_pd = fin.toPandas()
fin_pd = fin_pd.applymap(lambda x: str(x).split(".")[0])


display(fin_pd)

MatchID,Team1,Team1Goals,Team2Goals,Team2,MatchWinner,MatchStage
15,Germany,2,6,Belgium,Belgium,Final


In [0]:
#storing required data in df
fin_vis = pd.DataFrame(columns=['Fin_Teams', 'Fin_Score'])

#displaying all teams and respective scores in columns
for index, row in fin_pd.iterrows():
    team1 = row['Team1']
    team1goals = row['Team1Goals']
    team2 = row['Team2']
    team2goals = row['Team2Goals']
    fin_vis = pd.concat([fin_vis, pd.DataFrame({'Fin_Teams': [team1], 'Fin_Score': [team1goals]})], ignore_index=True)
    fin_vis = pd.concat([fin_vis, pd.DataFrame({'Fin_Teams': [team2], 'Fin_Score': [team2goals]})], ignore_index=True)

display(fin_vis)
     

Fin_Teams,Fin_Score
Germany,2
Belgium,6


In [0]:
fin_row_pd = pd.DataFrame({col: [''] for col in fin_vis.columns})
fin1 = fin_vis.iloc[:1]  # First value
fin2 = fin_vis.iloc[1:]  # Second value

for _ in range(8):
    fin1 = pd.concat([fin_row_pd, fin1], ignore_index=True)
    fin2 = pd.concat([fin_row_pd, fin2], ignore_index=True)

for _ in range(7):
    fin1 = pd.concat([fin1, fin_row_pd], ignore_index=True)
    fin2 = pd.concat([fin2, fin_row_pd], ignore_index=True)

lfin_new_column_name ={'Fin_Teams': 'L_Fin_Teams',
                       'Fin_Score': 'L_Fin_Score'
                       }
rfin_new_column_name ={'Fin_Teams': 'R_Fin_Teams',
                       'Fin_Score': 'R_Fin_Score'
                       }

fin1 = fin1.rename(columns=lfin_new_column_name)
fin2 = fin2.rename(columns=rfin_new_column_name)

rfin_new_column_order = ['R_Fin_Score', 'R_Fin_Teams']
fin2 = fin2[rfin_new_column_order]

display(fin1)
display(fin2)
     

L_Fin_Teams,L_Fin_Score
,
,
,
,
,
,
,
,
Germany,2
,


R_Fin_Score,R_Fin_Teams
,
,
,
,
,
,
,
,
6,Belgium
,


In [0]:
winner = spark.sql("SELECT MatchWinner FROM EliminationMatches WHERE MatchStage = 'Final'")
winner_pd = winner.toPandas()
winner_pd = winner_pd.applymap(lambda x: str(x).split(".")[0])

win_row_pd = pd.DataFrame({col: [''] for col in winner_pd.columns})

for _ in range(7):
    winner_pd = pd.concat([win_row_pd, winner_pd], ignore_index=True)

for _ in range(8):
    winner_pd = pd.concat([winner_pd, win_row_pd], ignore_index=True)

fin_col = {"MatchWinner":"TournamentWinner"}
winner_pd = winner_pd.rename(columns = fin_col)
display(winner_pd)

TournamentWinner
""
""
""
""
""
""
""
Belgium
""
""


In [0]:

# Concatenating DataFrames with keys
knockout_df = pd.concat([left_gs_spaced,
                        left_r16_spaced, 
                        left_qf_spaced, 
                        left_sf_spaced,
                        fin1,
                        winner_pd,
                        fin2,
                        right_sf_spaced,
                        right_qf_spaced,
                        right_r16_spaced,
                        right_gs_spaced
                         ], 
                        axis=1,
                        # keys=['left_gs', 'left_r16', 'left_qf', 'left_sf', 'fin1', 'winner', 'fin2', 'right_sf', 'right_qf', 'right_r16', 'right_gs']
                        )

# Displaying the concatenated DataFrame
display(knockout_df)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:467: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected bytes, got a 'int' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


L_Group,L_TeamRank,L_Country,L_P,L_W,L-D,L_L,L_GS,L_GC,L_GD,L_Pts,L_R16_Teams,L_R16_Score,L_QF_Teams,L_QF_Score,L_SF_Teams,L_SF_Score,L_Fin_Teams,L_Fin_Score,TournamentWinner,R_Fin_Score,R_Fin_Teams,R_SF_Score,R_SF_Teams,R_QF_Score,R_QF_Teams,R_R16_Score,R_R16_Teams,R_Group,R_TeamRank,R_Country,R_P,R_W,R-D,R_L,R_GS,R_GC,R_GD,R_Pts
A,1,France,3,2,1,0,12,8,4,7,,,,,,,,,,,,,,,,,,D,1,Sweden,3,2,1,0,10,4,6,7
A,2,Spain,3,2,0,1,10,10,0,6,Portugal,2,,,,,,,,,,,,,,1,Czech Republic,D,2,Russia,3,1,1,1,5,5,0,4
A,3,Germany,3,1,1,1,8,7,1,4,Germany,5,,,,,,,,,,,,,,2,England,D,3,Austria,3,1,0,2,4,6,-2,3
A,4,Italy,3,0,0,3,6,11,-5,0,,,,,,,,,,,,,,,,,,D,4,Denmark,3,1,0,2,4,8,-4,3
,,,,,,,,,,,,,Germany,6,,,,,,,,,,5,England,,,,,,,,,,,,,
,,,,,,,,,,,France,3,Turkey,6,,,,,,,,,,1,Finland,5,Sweden,,,,,,,,,,,
B,1,Portugal,3,2,1,0,12,7,5,7,Turkey,5,,,Germany,3,,,,,,2,England,,,5,Finland,E,1,Czech Republic,3,3,0,0,9,2,7,9
B,2,Belgium,3,2,0,1,11,8,3,6,,,,,,,,,Belgium,,,,,,,,,E,2,Wales,3,2,0,1,9,7,2,6
B,3,England,3,1,1,1,10,11,-1,4,,,,,,,Germany,2,,6,Belgium,,,,,,,E,3,Ukraine,3,1,0,2,7,8,-1,3
B,4,Netherlands,3,0,0,3,4,11,-7,0,North Macedonia,2,,,North Macedonia,1,,,,,,6,Belgium,,,0,Switzerland,E,4,Scotland,3,0,0,3,4,12,-8,0
